# HW5: Unsupervised Speech Recognition (USR)

Welcome to HW5 in Introduction to Deep Learning 11685. You will be working on Unsupervised Speech Recognition with GANs in this HW. You will be reimplementing and further improving on the model given in the USR paper by Facebook AI.<br>
Link: https://arxiv.org/abs/2105.11084


# Installations

In [ ]:
! pip install git+https://github.com/pytorch/fairseq # need to install fairseq
# You can install other libraries such as torchsummaryX, wandb and so on
! pip install torchsummaryX
! pip install wandb
! pip install -q torchsummaryX wandb --quiet
! pip install scikit-learn
! pip install matplotlib
! pip install tqdm
! pip install torch torchvision
! pip install fairseq
! pip install tensorboardX

  Cloning https://github.com/pytorch/fairseq to /tmp/pip-req-build-hdhuxh03
  Running command git clone --filter=blob:none --quiet https://github.com/pytorch/fairseq /tmp/pip-req-build-hdhuxh03
  Resolved https://github.com/pytorch/fairseq to commit da8fb630880d529ab47e53381c30ddc8ad235216
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 488.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 602.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=19974562 sha256=9b0eb73d4a3a7ffdcd88bf7376edde649e9ed1b7a4713

# Kaggle

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle/

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"adritadas","key":"4d8e5e73b5126973227f2e938ba1249f"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73249 sha256=d952caec15c28779622a4ae8c2fb32841e2799f647281a6bf246c3191fc93ae0
  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16


In [ ]:
!kaggle competitions download -c 11-685-s23-hw5
!mkdir '/content/data'

!unzip -qo '/content/11-685-s23-hw5.zip' -d '/content/data'

100% 4.85G/4.85G [00:58<00:00, 122MB/s]
100% 4.85G/4.85G [00:58<00:00, 89.6MB/s]


In [ ]:
!mkdir '/content/hw5_s23'

# Imports

In [ ]:
import torch
from torch import nn, optim
from torch.utils import data
from torch.nn.utils.rnn import *

import numpy as np
from tqdm import tqdm
import sys
import json

# add any other imports that you want

has_cuda = torch.cuda.is_available()
if has_cuda:
  print("GPU: ", torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if has_cuda else "cpu")
print("Device: ", device)

GPU:  Tesla V100-SXM2-16GB
Device:  cuda:0


In [ ]:
# TODO
%cd /content/hw5_s23
from model import Wav2vec_U

from task import UnpairedAudioText


/content/hw5_s23
GPU:  Tesla V100-SXM2-16GB


# Dataset and DataLoaders

You have TODOs which need to be completed in `task/unpaired_audio_text.py` before you run these cells. You just need to replace the paths. You can use the original code base as a reference.



In [ ]:
#from task import UnpairedAudioText
#task = UnpairedAudioText()
!python /content/drive/MyDrive/hw5_handout_S23/dataset/extracted_features_dataset.py
!python /content/drive/MyDrive/hw5_handout_S23/dataset/random_input_dataset.py

2023-08-11 04:42:48.001087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 04:42:51.982915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-11 04:43:08.761829: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 04:43:09.672217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import torch
from torch.utils import data

task = UnpairedAudioText()

In [ ]:
train_dataloader_args = dict(batch_size= 16, #feel free to change these values
                             #num_workers=2,
                            ) if has_cuda else dict(batch_size= 16)
train_dataloader_args["shuffle"] = True
train_dataloader_args["collate_fn"] = task.datasets["train"].collater

validation_dataloader_args = train_dataloader_args.copy()
validation_dataloader_args["shuffle"] = False
validation_dataloader_args["collate_fn"] = task.datasets["valid"].collater

train_dataloader = data.DataLoader(task.datasets["train"], **train_dataloader_args)
validation_dataloader = data.DataLoader(task.datasets["valid"], **validation_dataloader_args)

# Model and Training Configurations

You need to complete the TODOs in `model/wav2vec_u.py` before you run this cell. You can use the original codebase as a refernce to complete this.
Original Codebase: https://github.com/pytorch/fairseq/blob/main/examples/wav2vec/unsupervised/


In [ ]:
!python /content/drive/MyDrive/hw5_handout_S23/model/wav2vec_u.py

python3: can't open file '/content/drive/MyDrive/hw5_handout_S23/model/wav2vec_u.py': [Errno 2] No such file or directory


In [ ]:
from model import Wav2vec_U

model = Wav2vec_U(task.target_dictionary).to(device)
print(model)

Wav2vec_U(
  (discriminator): Discriminator(
    (net): Sequential(
      (0): Conv1d(66, 384, kernel_size=(6,), stride=(1,), padding=(5,))
      (1): SamePad()
      (2): Dropout(p=0.3, inplace=False)
      (3): Sequential(
        (0): Conv1d(384, 384, kernel_size=(6,), stride=(1,), padding=(5,))
        (1): SamePad()
        (2): Dropout(p=0.3, inplace=False)
        (3): GELU(approximate='none')
      )
      (4): Conv1d(384, 1, kernel_size=(6,), stride=(1,), padding=(5,))
      (5): SamePad()
    )
  )
  (segmenter): JoinSegmenter()
  (generator): Generator(
    (dropout): Dropout(p=0.1, inplace=False)
    (proj): Sequential(
      (0): TransposeLast()
      (1): Conv1d(512, 66, kernel_size=(4,), stride=(1,), padding=(2,), bias=False)
      (2): TransposeLast()
    )
  )
)


For a GAN, you need optimizers for both the discriminator and the generator. Configure the optimizers according to fairseq's configuration given in the link:
https://github.com/pytorch/fairseq/blob/main/examples/wav2vec/unsupervised/config/gan/w2vu.yaml


In [ ]:
!pip install apex

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user


In [ ]:
import torch.optim as optim
import itertools

# Assuming you have defined your model and its parameters as 'model', 'generator', and 'segmenter'
# Assuming you have specified the learning rate, weight decay, and betas as desired

# Create separate optimizers for discriminator, generator, and segmenter

from itertools import chain

num_epochs = 2000
epoch_start = 1

if epoch_start == 1:

  discriminator_optimizer = optim.Adam(model.discriminator.parameters(), lr=0.0005, weight_decay=0.0001, betas=(0.5, 0.98), eps= 1e-6)
  generator_optimizer = optim.Adam(
    itertools.chain(model.generator.parameters(), model.segmenter.parameters()),
    lr=0.0004, weight_decay=0, betas=(0.5, 0.98), eps = 1e-6
)

# Create the optimizer dictionary
optimizer = {
    "discriminator": discriminator_optimizer,
    "generator": generator_optimizer
}


A bunch of TODOs in the next cell. <br><br>
Tip: Instead of completing whole `run_model` function and the debugging while running the experiment section, you can create a new cell and code your own sanity check. It may help you to understand what is returned from the dataloader, what needs to be pushed to the device, how model is called and what `loss_stats` are.

In [ ]:
# Hint: You may find pdb to be a great tool in helping you understand returned values
# from the dataloader and the model. Usage:
# import pdb
# pdb.set_trace()

def run_model(model, dataloader):
    cumulative_stats = dict()
    #num_updates = 0

    for data in tqdm(dataloader, desc="Train" if model.training else "Eval "):
        #print("Net Input:", net_input)
        net_input = data['net_input']
        #print
        # What are the keys and values obtained from the data loader?
        # TODO: move all tensors to GPU
        # Tip: Checking what is inside net_input might help
        net_input = {k: v.to(device) for k,v in net_input.items()}

        if model.training:
            #num_updates += 1
            # TODO: We are training the model. Might need to do something with the optimizer?
            # Remember that you are training a GAN. Both optimizers won't be used at the same time.
            # You may have to write an if statement or something similar to use the specific optimizer.
            # You may have to use the discrim_step() attribute in the Wav2vec_U class
            if model.discrim_step(model.update_num):
                optimizer["discriminator"].zero_grad()

            else:
                optimizer["generator"].zero_grad()



            loss_stats = model(**net_input) # forward pass


            total_loss = 0.0

            for loss_name, loss_value in loss_stats["losses"].items():
                if loss_value is not None:
                    total_loss += loss_value


            # TODO: accumulate losses into total_loss for backprop during training
            # loss_stats["losses"] is a dictionary containing various loss components
            # some losses can be None if it's not used


            total_loss /= net_input["features"].size(0) # average by batch


            group = model.get_groups_for_update()
            # Look at what the get_groups_for_update() function does in the Wav2vec_U class
            # Can you try to think how you can use discrim_step() previously?

            # TODO: backprop loss and run the corresponding optimizer (Tip: See what 'group' is)

            total_loss.backward()
            optimizer[group].step()

        else:
            # validation
            loss_stats = task.valid_step(data, model)
            #print(loss_stats)


        # accumulate batch stats
        for k, v in loss_stats.items():
            if type(v) is dict:
                 # flatten inner dictionary
                key_value_pairs = [(k + "_" + kn, vn) for kn, vn in v.items()]
            else:
                key_value_pairs = [(k, v)]

          # TODO: accmulate all statistics into cumulative_stats, a dictionary
          # NOTE: you should convert any returned tensors to either values or numpy arrays
          # cumulative_stats shouldn't have nested dictionaries

            for key, value in key_value_pairs:
              if value is None :
                continue
              if isinstance(value, torch.Tensor):
                value = value.detach().cpu().numpy()

              if key not in cumulative_stats.keys():
                cumulative_stats[key] = value

              else:
                cumulative_stats[key] += value

    # average stats over the dataset
    # Note that some metrics are already averaged over batch, so the result won't make sense
    # You can fix them if needed
    for k, v in cumulative_stats.items():
        v = v / len(dataloader.dataset)
        if type(v) is np.ndarray:
            v = v.tolist()
        cumulative_stats[k] = v

    return cumulative_stats

In [ ]:
import wandb
import sys
wandb.login(key="e2fbcbd28e7f2e0bdeb1ab36f90d65664c335bd7")

In [ ]:
run = wandb.init(
    name    = "homework-5", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw5", ### Project should be created in your wandb account
    #config  = config ### Wandb Config for your run
)

edit_distance,▃█▅▅▂█▆▅▃▆▆▅▅▅▇▇▄▆▅▄▆▅▇▆▃▆▂▅▇▃▃▁▆▅▄▅▃▃▃▇
edit_distance,108.433


# Experiments

In [ ]:
print(f"Training for {num_epochs} epochs", file=sys.stderr)

eval_interval = 1 # evaluation after how many epochs?
best_eval_loss = 1e9

for epoch in range(epoch_start, num_epochs + 1):
    print(f"Epoch {epoch}", file=sys.stderr)

    model.train()
    # The model uses the epoch number to decide which part of the network to train
    model.set_num_updates(epoch) # Look at what this function does in the Wav2vec_U class

    train_stats = run_model(model, train_dataloader)

    if epoch % eval_interval == 0:

        model.eval()
        with torch.no_grad():
            eval_stats = run_model(model, validation_dataloader)
            print(eval_stats["edit_distance"])

        # TODO: perhaps save your model and optimizer here
        # Tip: You can even save the model after every epoch along with the best model.
        # This may help to continue training even if the best model is from a very early epoch.



        torch.save(model.state_dict(), f"model_epoch_{epoch}.pt")
        #torch.save(optimizer.state_dict(), f"optimizer_epoch_{epoch}.pt")
        wandb.log({"edit_distance": eval_stats["edit_distance"]})



    else:
        eval_stats = {}

    # TODO: Log training/eval statistics


wandb.finish()

# V100

Training for 2000 epochs
Epoch 1
Eval : 100%|██████████| 63/63 [00:01<00:00, 32.15it/s]
Epoch 2


131.081


Eval : 100%|██████████| 63/63 [00:01<00:00, 35.49it/s]
Epoch 3


103.632


Eval : 100%|██████████| 63/63 [00:01<00:00, 35.83it/s]
Epoch 4


...

Eval : 100%|██████████| 63/63 [00:01<00:00, 35.53it/s]
Epoch 2000


108.114


Eval : 100%|██████████| 63/63 [00:01<00:00, 35.54it/s]


107.936


edit_distance,▆▄▆▅▄█▃▄▄▃▆▄▄▁▅▅▆▄▃▄▅▄▃▂▁▄▃▂▃▃▄▅▅▄▅▄▅▃▅▄
edit_distance,107.936


In [ ]:
print(eval_stats)

{'edit_distance': 107.936, '_num_chars': 125.089, '_num_pred_chars': 77.321, 'nsentences': 1.0, '_world_size': 0.063, '_lm_score_sum': 0.0, '_kaldi_score_sum': 0.0, '_word_lm_sum': 0.0, '_num_words': 0.0, '_vocab_seen': [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0]}


# Testing and submission to Kaggle

In [ ]:
from dataset import extracted_features_dataset
test_path = # TODO: PathToTest
testset = extracted_features_dataset.ExtractedFeaturesDataset(path=test_path,
                                                              split='test')

test_dataloader_args = train_dataloader_args.copy()
test_dataloader_args["shuffle"] = False
test_dataloader_args["collate_fn"] = testset.collater

test_dataloader = data.DataLoader(testset, **test_dataloader_args)

SyntaxError: ignored

Write the `test_step` function which can be coded very similar to `valid_step` given in `task/unpaired_audio_text.py`

In [ ]:
def test_step(self, inputs, model):
    sample = inputs

    res = model(
        **sample["net_input"],
        dense_x_only=True,
    )

    dense_x = res["logits"]
    padding_mask = res["padding_mask"]

    word_scores = None
    if self.compute_word_score is not None:
        word_scores = self.compute_word_score(dense_x.cpu(), padding_mask.cpu())

    z = dense_x.argmax(-1)
    z[padding_mask] = self.target_dictionary.pad()

    vocab_seen = torch.zeros(self.num_symbols, dtype=torch.bool)

    c_err = 0
    c_len = 0
    pred_c_len = 0
    lm_score_sum = 0

    for i, (x, t, id) in enumerate(
        zip(
            z,
            sample["target"] if "target" in sample else [None] * len(z),
            sample["id"],
        )
    ):

        if t is not None:
            t = t[(t >= self.target_dictionary.nspecial)]
        x = x[
            (x >= self.target_dictionary.nspecial)
            & (x < (self.num_symbols + self.target_dictionary.nspecial))
        ]
        if self.sil_id >= 0:
            x = x[x != self.sil_id]

        vocab_seen[x - self.target_dictionary.nspecial] = True

        pred_units_arr = x
        if self.cfg.ctc_eval:
            pred_units_arr = pred_units_arr.unique_consecutive()
            pred_units_arr = pred_units_arr[pred_units_arr != 0]

        if id == 0:
            if t is not None:
                logger.info(f"REF: {self.target_dictionary.string(t)}")
            logger.info(f"HYP: {self.target_dictionary.string(pred_units_arr)}")

            if self.kenlm is not None:
                if t is not None:
                    ref_lm_s = self.compute_lm_score(
                        self.target_dictionary.string(t)
                    )
                    logger.info(
                        f"LM [REF]: {ref_lm_s}, {math.pow(10, -ref_lm_s / (len(t) + 1))}"
                    )

                hyp_lm_s = self.compute_lm_score(
                    self.target_dictionary.string(pred_units_arr)
                )
                logger.info(
                    f"LM [HYP]: {hyp_lm_s}, {math.pow(10, -hyp_lm_s / (len(pred_units_arr) + 1))}"
                )

        pred_units_arr = pred_units_arr.tolist()

        pred_c_len += len(pred_units_arr)

        if t is not None:
            t = t.tolist()
            c_err += editdistance.eval(pred_units_arr, t)
            c_len += len(t)
        else:
            c_len = pred_c_len

        if self.kenlm is not None:
            pred_str = self.target_dictionary.string(pred_units_arr)
            lm_score = self.compute_lm_score(pred_str)  # Call compute_lm_score on the pred_str
            lm_score_sum += lm_score

    kaldi_score_sum = 0
    word_lm_sum = 0
    num_words = 0
    if word_scores is not None:
        for score, words in word_scores:
            kaldi_score_sum += score
            num_words += len(words)
            if self.word_kenlm is not None:
                word_lm_sum += self.kenlm.score(" ".join(words))

    try:
        world_size = get_data_parallel_world_size()
    except:
        world_size = 1

    logging_output = {
        "edit_distance": c_err,
        "_num_chars": c_len,
        "_num_pred_chars": pred_c_len,
        "nsentences": z.size(0),
        "_world_size": world_size,
        "_lm_score_sum": lm_score_sum,
        "_kaldi_score_sum": kaldi_score_sum,
        "_word_lm_sum": word_lm_sum,
        "_num_words": num_words,
        "_vocab_seen": vocab_seen,
    }

    return logging_output


Write some code to evaluate and get the results. You are free to write the below cells however you want

In [ ]:
model.eval()
results = []

for data in tqdm(test_dataloader, desc="Test"):
    # TODO
    pass

In [ ]:
# TODO: Replace the path and get the phoneme_map.json for mapping
with open("/Path/To/phoneme_map.json", "r") as file:
    phon_map = json.load(file)

In [ ]:
predictions = []

for line in results:
    # TODO: Map results with phon_map
    pass

In [ ]:
# TODO: Make the CSV and submit to kaggle